## * 데이터 준비

In [5]:
import nltk
# nltk.download("punkt")
# nltk.download('punkt_tab')
# nltk.download('averaged_perceptron_tagger_eng')
# nltk.download('stopwords')

In [6]:
import pandas as pd

In [9]:
df = pd.read_csv("D:\강의\텍스트마이닝(16시간)\영어 크롤링.csv")
df

,Date,Name,Score,Review,like
0,"January 26, 2020",Bernard Reis,1,Completely frustrating experience. Paid extra ...,7
1,"January 18, 2020",Gordon Inman,2,"Some good features, but this app isn't very sm...",8
2,"January 18, 2020",MrDurma,4,"This app works perfectly, it allowed us to dow...",2
3,"January 26, 2020",K Kostas,3,Good but not great for an app by LG. Not the m...,1
4,"January 17, 2020",Greg Evans,3,App is useful but limited and requires some up...,1
...,...,...,...,...,...
7145,"December 31, 2019",Mr Darryl Maurice Friend,5,Wonderful,0
7146,"January 12, 2020",luis arias,5,Wonderful,0
7147,"September 24, 2017",Tobiloba Bakare,5,Awesome 🌟 🌟 🌟 🌟,0
7148,"October 13, 2017",A Google user,5,Good Good Good,0


In [11]:
stopwords = nltk.corpus.stopwords.words('english') #불용어

In [13]:
def pos_tagging(string):
    string = string.lower() #소문자변환
    string = nltk.word_tokenize(string) 
    string = nltk.pos_tag(string)
    result = [word for word, tag in string if tag in {"NN", "NNS", "NNP", "NNPS"} if word not in stopwords]
    return result

In [15]:
#nltk tagset보기
nltk.download('tagsets')
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [17]:
from tqdm import tqdm

In [19]:
tagged_pos_document = []
for i in tqdm(df['Review']):
    tagged_pos_document.append(pos_tagging(i))

100%|██████████| 7150/7150 [00:06<00:00, 1038.17it/s]


In [20]:
tagged_pos_document[0]

['experience',
 'money',
 'air',
 'conditioner',
 'connectivity',
 'times',
 'device',
 'minute',
 'app',
 'cache',
 'router',
 'router',
 'nothing',
 'works']

## 1. Word2Vec
- vector_size=100 → 단어 벡터의 차원 수 (일반적으로 논문에서 100~300사용)
- window=2 → 중심 단어 기준으로 앞뒤 2개 단어를 컨텍스트로 사용
- min_count=1 → 최소 1번 이상 등장한 단어만 학습
- sg → 학습방식 (0이면 CBOW, 1이면 Skip-gram)

In [29]:
!pip install gensim

In [1]:
from gensim.models import Word2Vec

In [21]:
w2v_model = Word2Vec(tagged_pos_document, vector_size=100, window=2, min_count=1, sg = 0)

In [25]:
w2v_model.wv.key_to_index.keys()

dict_keys(['app', 'lg', 'phone', 'washer', 'time', 'machine', 'dryer', 'permissions', 'cycle', 'access', 'device', 'wifi', 'update', 'tv', 'account', 'location', 'product', 'home', 'cycles', 'fridge', 'ac', 'appliances', 'work', 'works', 'times', 'error', 'google', 'appliance', 'notifications', 'way', 'problem', '..', 'air', 'laundry', 'feature', 'nothing', 'devices', 'start', 'data', 'information', 'refrigerator', 'washing', 'settings', 'network', 'connection', 'calls', 'option', 'diagnosis', 'application', 'issue', 'features', 'temperature', 'load', 'unit', 'support', 'control', 'products', 'permission', 'apps', 'dishwasher', 'screen', 'notification', 'wash', 'reason', 'use', 'function', 'options', 'thing', 'everything', 'waste', 'version', 'issues', 'point', 'tag', 'nfc', 'program', 'button', 'conditioner', 'anything', 'functionality', 'thanks', 'service', 'samsung', 'info', 'registration', 'customer', 'thinq', 'lot', 'useless', 'garbage', '*', 'message', 'status', 'country', 'thing

In [27]:
w2v_model.wv["app"] #벡터값 추출

array([-0.14322704,  0.5244759 , -0.1305257 ,  0.1056385 ,  0.06478437,
       -0.53018737,  0.2018973 ,  0.91856354, -0.3994882 , -0.14562528,
       -0.15478611, -0.30846804, -0.22364517,  0.21737796,  0.04743551,
       -0.35650042,  0.08205788, -0.4656468 , -0.08327043, -0.8288677 ,
        0.09025341,  0.18252881,  0.20859289, -0.30096075, -0.0467227 ,
        0.06477669, -0.42211217, -0.3492854 , -0.2428312 ,  0.15814751,
        0.43877444,  0.0352757 ,  0.35709321, -0.28177434, -0.12745203,
        0.30306754, -0.03513848, -0.3265557 ,  0.19523296, -0.81116146,
        0.04682019, -0.3133272 ,  0.05172738, -0.02636699,  0.40225315,
       -0.11269527, -0.11035728, -0.09733921,  0.15360619,  0.24764223,
        0.1571087 , -0.25332192, -0.1012031 , -0.10730069, -0.17946133,
        0.10914197,  0.5059913 ,  0.07214756, -0.46521804,  0.21462533,
        0.27077544,  0.13574864, -0.16471589, -0.02135078, -0.49757248,
        0.4713933 ,  0.32041067,  0.18835835, -0.5780038 ,  0.50

In [29]:
w2v_model.wv.most_similar("app", topn=10)

[('wifi', 0.9995769262313843),
 ('data', 0.9995689988136292),
 ('people', 0.9995607733726501),
 ('way', 0.999560534954071),
 ('lg', 0.9995591640472412),
 ('point', 0.9995476603507996),
 ('appliance', 0.9995332360267639),
 ('devices', 0.9995325207710266),
 ('appliances', 0.9995175004005432),
 ('apps', 0.9995115399360657),
 ('update', 0.9995102286338806),
 ('response', 0.9995080828666687),
 ('unit', 0.999504029750824),
 ('ac', 0.999502420425415),
 ('nothing', 0.9995015859603882),
 ('issue', 0.9994946718215942),
 ('product', 0.999491810798645),
 ('problem', 0.999488890171051),
 ('device', 0.9994677305221558),
 ('feature', 0.9994636178016663)]

## 2. Doc2Vec
- vector_size: 문서 벡터의 차원
- alpha : 모델 학습시 초기학습률
- min_alpha: alpha가 이 값 이하로 감소하지 않도록 제한하는 값
- window: 주변 단어를 고려하는 범위
- min_count: 단어 최소 등장 횟수 
- epochs: 학습 반복 횟수
- dm: 1은 PV-DM, 0은 PV-DBOW

In [32]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [34]:
tagged_pos_document[1]

['features',
 'app',
 'device',
 'ui/ux',
 'device',
 'image',
 'stove',
 'stovetop',
 'burner',
 'layout',
 'directions',
 'start',
 'product',
 'look',
 'effort',
 'experience',
 'device',
 'rate']

In [36]:
tagged_corpus_list = []

for i, doc in enumerate(tagged_pos_document):
  tag = 'document{}'.format(i)
  tagged_corpus_list.append(TaggedDocument(tags = [tag], words = doc))

In [38]:
tagged_corpus_list[1]

TaggedDocument(words=['features', 'app', 'device', 'ui/ux', 'device', 'image', 'stove', 'stovetop', 'burner', 'layout', 'directions', 'start', 'product', 'look', 'effort', 'experience', 'device', 'rate'], tags=['document1'])

In [42]:
#모델 선언
doc2vec_model = Doc2Vec(vector_size=300,
                        alpha = 0.025,
                        min_alpha = 0.01, #알파값이 최소값 이하로 감소하지 않도록 제한
                        window = 4,
                        min_count = 1,
                        epochs= 100,
                        dm = 1
                        )

In [44]:
#단어사전 생성
doc2vec_model.build_vocab(tagged_corpus_list)

In [46]:
#학습
doc2vec_model.train(tagged_corpus_list, #학습할 데이터
                    total_examples=doc2vec_model.corpus_count, #모델에서 학습할 전체 문서의 수
                    epochs=doc2vec_model.epochs #doc2vec모델에서 사용했던
                    )

In [47]:
#문서간의 유사도
doc2vec_model.dv.most_similar("document0", topn=20)

[('document2476', 0.6592808961868286),
 ('document3232', 0.6436310410499573),
 ('document6619', 0.6079527139663696),
 ('document602', 0.6048237085342407),
 ('document991', 0.5696893930435181),
 ('document137', 0.5682743787765503),
 ('document2087', 0.5663574934005737),
 ('document4257', 0.5631917715072632),
 ('document2981', 0.5627456903457642),
 ('document3327', 0.559985339641571),
 ('document3777', 0.5558426976203918),
 ('document3044', 0.5549496412277222),
 ('document3325', 0.5546692609786987),
 ('document2371', 0.553124725818634),
 ('document4060', 0.552293062210083),
 ('document1456', 0.5518935322761536),
 ('document6192', 0.5518625974655151),
 ('document529', 0.551253616809845),
 ('document2673', 0.5512313842773438),
 ('document1849', 0.5503976345062256)]

In [48]:
#새로운 문서에 대한 벡터값 만들기
new_doc_vec = doc2vec_model.infer_vector(['lg', 'app', 'is', 'good'])
new_doc_vec

array([-0.12395682, -0.17719002, -0.06275748,  0.05089861, -0.06553577,
       -0.08992471,  0.02172406,  0.37645516, -0.07621656,  0.05385873,
       -0.12851103, -0.03005419, -0.02399445,  0.1038911 , -0.01075204,
       -0.11475102,  0.06591657, -0.03989733,  0.18977682,  0.01709971,
        0.01255459, -0.06606177, -0.09003537, -0.26618752, -0.12481942,
        0.12975481, -0.33818367,  0.04364847,  0.32999372,  0.17159998,
        0.23052873,  0.10684828,  0.08029971,  0.08314615,  0.13500826,
        0.22018054, -0.24761532, -0.20896526,  0.0421971 ,  0.02262196,
       -0.0031709 , -0.08701433, -0.10975395, -0.02443489,  0.14631546,
        0.21399553,  0.10721312, -0.07054471,  0.22427559,  0.20363139,
        0.20837046, -0.05169185,  0.19306462,  0.08720171, -0.09285246,
        0.09758543, -0.07866235, -0.11185064, -0.03723781,  0.09701414,
       -0.04563029, -0.12854616, -0.15985607,  0.14896826, -0.08166856,
        0.19473761,  0.25635612,  0.10024362, -0.07124674,  0.00

In [49]:
doc2vec_model.wv.similarity('app', 'lg')

0.4599667

## 3. FastText

In [54]:
from gensim.models import FastText
from gensim.models.fasttext import FastText as FT

In [56]:
tagged_pos_docuemnt

NameError: name 'tagged_pos_docuemnt' is not defined

In [ ]:
# FastText 모델
model_fasttext = FT(vector_size=300, window=2, min_count=1, epochs=100)

In [ ]:
#학습 데이터로 모델 학습
model_fasttext.build_vocab(tagged_pos_docuemnt)

In [ ]:
#모델학습
model_fasttext.train(tagged_pos_docuemnt, total_examples=model_fasttext.corpus_count, epochs=model_fasttext.epochs)

In [ ]:
model_fasttext.wv.most_similar('lg', topn=5)

In [ ]:
#유사도는 -1에서 1 
# 1에 가까울수록 단어가 유사하다
model_fasttext.wv.similarity('lg', 'app')